# Limits

## 1. Resources limits

### 1.1. User resources limits

- The main config file is: `/etc/security/limits.conf`
- Also include all `.conf` file in `/etc/security/limits.d/`

In [ ]:
echo "* limits config are: "
cat /etc/security/limits.conf

#### 1.1.1. `ulimit` command

In [ ]:
echo '* current user resources limit is: '
ulimit

echo -e '\n* hard resources with open file(-n) and max process(-u) limits are: '
ulimit -H -n -u

echo -e '\n* soft resources with open file(-n) and max process(-u) limits are: '
ulimit -S -n -u

echo -e '\n* all resources limit is: '
ulimit -a

### 1.2. Service resources limits

- For system services, the config files include `/etc/systemd/system.conf` and all files as `/etc/systemd/system.conf.d/*.conf`
- For user services,  the config files include `/etc/systemd/user.conf` and all files as `/etc/systemd/user.conf.d/*.conf`

In [ ]:
echo '* system services resources limit is: '
cat /etc/systemd/system.conf

echo -e '\n* user services resources limit is: '
cat /etc/systemd/user.conf

#### 1.2.1. Get service resources limits

In [ ]:
echo -e '* the resources limits of docker service are: '
cat /proc/$(cat /run/docker.pid)/limits

- The content format is:
    ```
    {user}   {soft/hard}   {resource_type}   {limit_value}
    ```
- 'soft' is warning value, 'hard' is max value

### 1.3. Process limits

#### 1.3.1. User process limits

- Edit `/etc/security/limits.d/20-nproc.conf` file
- Set process limit for user

```ini
# Default limit for number of user's processes to prevent
# accidental fork bombs|.
# See rhbz #432903 for reasoning.

* soft nproc 2048
* hard nproc 4096

# Set process limits for root user
root soft nproc 65535
root hard nproc 65535
```

In [ ]:
echo '* The user process limits config are: '
cat /etc/security/limits.d/20-nproc.conf | grep nproc

#### 1.3.2. Service process limits

- Edit `/etc/systemd/system.conf` (or `/etc/systemd/user.conf`) file
- Set `DefaultLimitNPROC=65535`
- Reload systemd
- Restart service

In [ ]:
echo '* The system service process limits config is: '
cat /etc/systemd/system.conf | grep DefaultLimitNPROC

# Set DefaultLimitNPROC=65535

echo -e '\n* reload systemd: '
sudo systemctl daemon-reload

echo -e '\n* restart service: '
systemctl restart docker.service

#### 1.3.3. System process limits

##### 1.3.3.1. Show system process limits

In [ ]:
echo '* the system process limits: '
cat /proc/sys/kernel/pid_max

echo -e '\n* the system process limits: '
sysctl kernel.pid_max

##### 1.3.3.2. Modify system process limits

- Temporary modifications

In [ ]:
sudo echo 65535 > /proc/sys/kernel/pid_max

- Forever modifications

In [ ]:
sudo echo "kernel.pid_max=65535" >> /etc/sysctl.conf

### 1.4. File handle limits

#### 1.4.1. User file handle limits

- Edit `/etc/security/limits.conf` file
- Set nofile limit for user

```ini
# Default limit for number of user's processes to prevent
# accidental fork bombs.
# See rhbz #432903 for reasoning.

* soft nofile 65535 
* hard nofile 65535
```

In [ ]:
echo '* the user open file limits are:'
cat /etc/security/limits.conf | grep nofile

#### 1.4.2. Service file handle limits

- Edit `/etc/systemd/system.conf` (or `/etc/systemd/user.conf`) file
- Set `DefaultLimitNOFILE=65535`
- Reload systemd
- Restart service

In [ ]:
echo '* modify system service open file limits: '
sudo sed -i "s/#DefaultLimitNOFILE=/DefaultLimitNOFILE=655350/g" /etc/systemd/system.conf

echo '* the system service open file limits are: '
cat /etc/systemd/system.conf | grep DefaultLimitNOFILE

#### 1.4.3. System file handle limits

In [ ]:
echo '* modify system open file limits: '
echo 6553560 > /proc/sys/fs/file-max

echo -e '\n* the system open file limits is: '
cat /proc/sys/fs/file-nr